In [1]:
import urllib2
import urllib
import json
import pprint

# Scrape the first 10000 packages from data.gov relating to health
start = 0
rows = 1000
package_names = []
while start < 10000:
    response = urllib2.urlopen('https://catalog.data.gov/api/3/search/dataset?q=health&sort=score+desc&start=' + str(start) + '&rows=' + str(rows))
    package_names.extend(json.loads(response.read())["results"])
    start += rows
package_names

[u'health-areas',
 u'inspections-cac7e',
 u'mongolia-health',
 u'deficiencies-02c11',
 u'hpsa-mental-health',
 u'teale-health-facilities',
 u'public-health-units',
 u'community-health-survey',
 u'health-and-human-services-locations',
 u'adult-mental-health-treatment',
 u'health-care-cost-growth',
 u'associated-health-allocation',
 u'health-data-repository-hdr',
 u'home-health-agencies-3a23b',
 u'health-information-updates-d963c',
 u'mental-health-wait-times-d5beb',
 u'rural-health-clinics-c972f',
 u'public-health-clinics-308e0',
 u'public-health-department-events',
 u'health-region-polygon',
 u'health-region-polygonc7589',
 u'community-health-centers-48f3e',
 u'public-health-nursing-listing-6d805',
 u'community-health-center-point',
 u'rural-health-clinic-point',
 u'local-health-unit-point',
 u'health-facility-services-point',
 u'veterans-health-administration-behavioral-health-data',
 u'lesotho-health-project',
 u'find-a-health-center',
 u'home-health-compare-7413f',
 u'health-data-in

In [19]:
# Double checking they're all unique
print(len(package_names) == len(set(package_names)))
print(len(package_names))

True
10000


In [57]:
# Get the metadetails for these packages and store it into a list for processing. This will be gigantic.
data_mappings = {}
for package in package_names[:8000]:
    response = urllib2.urlopen("https://catalog.data.gov/api/3/action/package_show?id=" + package)
    data_mappings[package] = json.loads(response.read())["result"]


HTTPError: HTTP Error 403: Forbidden

In [58]:
# 0 to 1000
import pickle
def save_obj(obj, name ):
    with open(name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)
save_obj(data_mappings, "storage")
        

In [2]:
import pickle
def load_obj(name):
    with open(name + '.pkl', 'rb') as f:
        return pickle.load(f)
storage = load_obj("storage")

In [64]:
len(storage.keys())
storage[storage.keys()[0]]["resources"]

[{u'archiver': {u'cache_filepath': u'None',
   u'cache_url': u'None',
   u'created': u'2017-11-18T09:57:00.378465',
   u'etag': u'W/"acd3adc99106c0fa0c0bef4524c32066"',
   u'failure_count': 0,
   u'first_failure': None,
   u'hash': u'None',
   u'is_broken': False,
   u'is_broken_printable': u'Downloaded OK',
   u'last_modified': u'Mon, 27 Mar 2017 18:39:16 UTC',
   u'last_success': u'2017-11-18T09:57:00.362450',
   u'mimetype': u'text/csv',
   u'reason': u'',
   u'resource_timestamp': u'2017-01-05T07:48:23.650719',
   u'size': u'3',
   u'status': u'Archived successfully',
   u'status_id': 0,
   u'updated': u'2017-11-18T09:57:00.362450',
   u'url_redirected_to': None},
  u'cache_last_updated': None,
  u'cache_url': None,
  u'created': u'2016-11-24T04:06:40.971744',
  u'description': u'',
  u'format': u'CSV',
  u'hash': u'',
  u'id': u'99f801eb-ce98-4067-a1d3-160d967f52da',
  u'last_modified': None,
  u'mimetype': u'text/csv',
  u'mimetype_inner': None,
  u'name': u'Comma Separated Value

In [44]:
z = storage
half_of_packages = package_names[:6478] #Broke at 6478 but good enough for now
useful_shit = []
def has_csv(package, z):
    for shit in z[package]["resources"]:
        if ".csv" in shit["url"]:
            return shit
    return None
for package in half_of_packages:
    if has_csv(package, z):
        lmao = has_csv(package, z)
        useful_shit.append({"Filename": (lmao["name"] + z[package]["notes"] + lmao["description"])[:100], "Data Source": lmao["url"], "Relevant Articles": [lmao["description"],z[package]["notes"]]})

In [45]:
useful_shit

[{'Data Source': u'https://data.cityofnewyork.us/api/views/bg38-veix/rows.csv?accessType=DOWNLOAD',
  'Filename': u'Comma Separated Values FileGIS data: Boundaries of Health Areas',
  'Relevant Articles': [u'', u'GIS data: Boundaries of Health Areas']},
 {'Data Source': u'https://data.louisvilleky.gov/sites/default/files/Health_Inspections.csv',
  'Filename': u'Health Inspections<p>Attributes of routine and complaint driven inspections performed by Louisville ',
  'Relevant Articles': [u'Health_Inspections.csv',
   u'<p>Attributes of routine and complaint driven inspections performed by Louisville Metro Department of Public Health and Wellness.\xa0 EstablishmentID column can be joined to the EstablishmentID column in the Establishments Table to show attributes of the Establishment when a regulated establishment is involved.</p>\n']},
 {'Data Source': u'https://data.medicare.gov/api/views/r5ix-sfxw/rows.csv?accessType=DOWNLOAD',
  'Filename': u'https://data.medicare.gov/api/views/r5ix-s

In [46]:
len(useful_shit)

896

In [53]:
import csv
import urllib2
import sys
from elasticsearch import Elasticsearch, helpers

csv.field_size_limit(sys.maxsize)

def convert_csv_to_json(csv_url):
    response = urllib2.urlopen(csv_url)
    cr = csv.DictReader(response)
    return (list(cr))

def format_es_document(index, document):
    #print(filter(str.isalnum, str(index.lower().replace(" ", ""))))
    return {
        "_index": filter(str.isalnum, str(index.lower().replace(" ", ""))), # Must be lowercase
        "_type": filter(str.isalnum, str(index.lower().replace(" ", ""))),
        "_source": document
    }

es = Elasticsearch("https://search-data-pipeline-poc-bdfr3wal5lxncg2zllpoo6nd2e.us-east-1.es.amazonaws.com")

def chunks(l, n):
    """Yield successive n-sized chunks from l."""
    for i in range(0, len(l), n):
        yield l[i:i + n]

tmp = chunks(useful_shit, 20)
i = 0
for chunk in tmp:
    if i < 24:
        i+=1
    else:    
        for data in chunk:
            es_documents_for_datasets = []
            try:
                raw_documents = convert_csv_to_json(data["Data Source"])
                for raw_document in raw_documents:
                    es_documents_for_datasets.append(format_es_document(data["Filename"], raw_document))
            except Exception as e:
                print(e)
                continue
            print("Uploading documents...")
            #     res = es.index(index=data["Filename"],doc_type='post', body=document)
            #     print(res['result'])
            try:
                #print()
                print(helpers.bulk(es, es_documents_for_datasets))
            except Exception as e:
                print(e)
                continue

Uploading documents...
(209, [])
Uploading documents...
(1, [])
HTTP Error 404: Not Found
Uploading documents...
(19, [])
HTTP Error 404: Not Found
Uploading documents...
(32, [])
Uploading documents...
(209, [])
Uploading documents...
(290, [])
Uploading documents...
ConnectionTimeout caused by - ReadTimeoutError(HTTPSConnectionPool(host='search-data-pipeline-poc-bdfr3wal5lxncg2zllpoo6nd2e.us-east-1.es.amazonaws.com', port=443): Read timed out. (read timeout=10))
Uploading documents...
'ascii' codec can't decode byte 0xe2 in position 21: ordinal not in range(128)
Uploading documents...
'ascii' codec can't decode byte 0xe2 in position 21: ordinal not in range(128)
Uploading documents...
(66, [])
Uploading documents...
'ascii' codec can't decode byte 0xc2 in position 298: ordinal not in range(128)
Uploading documents...
'ascii' codec can't decode byte 0xe2 in position 255: ordinal not in range(128)
Uploading documents...
'ascii' codec can't decode byte 0xe2 in position 21: ordinal not i

Uploading documents...
(101030, [])
Uploading documents...
'ascii' codec can't decode byte 0xc2 in position 695: ordinal not in range(128)
Uploading documents...
(35642, [])
Uploading documents...
(35642, [])
Uploading documents...
(134203, [])
Uploading documents...
(1068, [])
Uploading documents...
(101030, [])
Uploading documents...
'ascii' codec can't decode byte 0xc2 in position 298: ordinal not in range(128)
Uploading documents...
ConnectionTimeout caused by - ReadTimeoutError(HTTPSConnectionPool(host='search-data-pipeline-poc-bdfr3wal5lxncg2zllpoo6nd2e.us-east-1.es.amazonaws.com', port=443): Read timed out. (read timeout=10))
Uploading documents...
(5834, [])
Uploading documents...
(367, [])
Uploading documents...
'ascii' codec can't decode byte 0x92 in position 655: ordinal not in range(128)
Uploading documents...
(3484, [])
Uploading documents...
(3551, [])
Uploading documents...
'ascii' codec can't decode byte 0xc2 in position 65: ordinal not in range(128)
Uploading documents

Uploading documents...
(7, [])
Uploading documents...
(34, [])
Uploading documents...
'ascii' codec can't decode byte 0xc2 in position 203: ordinal not in range(128)
Uploading documents...
(393, [])
Uploading documents...
(1305, [])
Uploading documents...
'ascii' codec can't decode byte 0xc3 in position 363: ordinal not in range(128)
HTTP Error 404: Not Found
Uploading documents...
(3, [])
Uploading documents...
(410, [])
'ascii' codec can't encode character u'\xae' in position 12: ordinal not in range(128)
Uploading documents...
(7, [])
Uploading documents...
'ascii' codec can't decode byte 0xc3 in position 156: ordinal not in range(128)
Uploading documents...
(6426, [])
Uploading documents...
(8604, [])
Uploading documents...
(42704, [])
Uploading documents...
'ascii' codec can't decode byte 0xef in position 120: ordinal not in range(128)
Uploading documents...
'ascii' codec can't decode byte 0xef in position 2: ordinal not in range(128)
Uploading documents...
(2928, [])
Uploading do

Uploading documents...
(288, [])
HTTP Error 404: Not Found
Uploading documents...
(16, [])
Uploading documents...
(211, [])
HTTP Error 404: Not Found
Uploading documents...
(262, [])
Uploading documents...
(3, [])
Uploading documents...
(7, [])
HTTP Error 404: Not Found
HTTP Error 500: Server Error
Uploading documents...
(13, [])
Uploading documents...
(112, [])
HTTP Error 404: Not Found
Uploading documents...
'ascii' codec can't decode byte 0xc2 in position 126: ordinal not in range(128)
Uploading documents...
(9599, [])
Uploading documents...
(11, [])
HTTP Error 404: Not Found
HTTP Error 404: Not Found
Uploading documents...
(6, [])
'ascii' codec can't encode character u'\u2018' in position 61: ordinal not in range(128)
HTTP Error 404: Not Found
Uploading documents...
(4, [])
Uploading documents...
(513, [])
Uploading documents...
(16, [])
Uploading documents...
(4, [])
Uploading documents...
(171, [])
Uploading documents...
'ascii' codec can't decode byte 0xc2 in position 654: ordin

HTTP Error 404: Not Found
Uploading documents...
(50, [])
Uploading documents...
(31, [])
'ascii' codec can't encode character u'\u2013' in position 42: ordinal not in range(128)
HTTP Error 404: Not Found
Uploading documents...
ConnectionTimeout caused by - ReadTimeoutError(HTTPSConnectionPool(host='search-data-pipeline-poc-bdfr3wal5lxncg2zllpoo6nd2e.us-east-1.es.amazonaws.com', port=443): Read timed out. (read timeout=10))
Uploading documents...
ConnectionTimeout caused by - ReadTimeoutError(HTTPSConnectionPool(host='search-data-pipeline-poc-bdfr3wal5lxncg2zllpoo6nd2e.us-east-1.es.amazonaws.com', port=443): Read timed out. (read timeout=10))
HTTP Error 404: Not Found
Uploading documents...
'ascii' codec can't decode byte 0xc2 in position 739: ordinal not in range(128)
HTTP Error 404: Not Found
HTTP Error 404: Not Found
HTTP Error 404: Not Found
Uploading documents...
(22, [])
HTTP Error 404: Not Found
HTTP Error 404: Not Found
HTTP Error 404: Not Found
Uploading documents...
(16605, [

In [ ]:
print(3)